# Transcriptomic profile estimation with all samples

In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats
from scipy import io

from sklearn.metrics.pairwise import cosine_similarity

import sys
sys.path.append("../../scripts")
from noiseReductionMethodology import preprocessing, Raman_model
from analysis_pclda import LDA_model
from predictFunc import calcPrediction
from util import returnValues

## Load data

In [2]:
RAMAN = pd.read_csv("../../data/RAMAN_FINGERPRINT.csv")
RAMAN_PROCESSED = preprocessing(RAMAN)
RAMAN_PROCESSED.head()

,0,1,2,3,4,5,6,7,8,9,...,590,591,592,593,594,595,596,597,598,label
0,0.777427,0.798224,0.788404,0.760493,0.758397,0.730189,0.688064,0.620933,0.629037,0.658134,...,-1.786467,-1.778755,-1.773075,-1.801913,-1.817556,-1.802574,-1.790176,-1.800517,-1.835198,0
1,0.819071,0.838680,0.801314,0.734114,0.704192,0.677671,0.615264,0.532932,0.525684,0.557399,...,-1.942792,-1.988656,-2.001273,-2.014088,-2.021082,-2.015529,-1.991117,-1.972676,-1.962001,0
2,0.784357,0.799093,0.757752,0.689691,0.634893,0.599651,0.571955,0.523794,0.562298,0.624879,...,-1.856136,-1.828801,-1.792533,-1.794846,-1.801112,-1.815291,-1.843331,-1.893848,-1.902506,0
3,0.911177,0.947830,0.915633,0.857712,0.857826,0.800391,0.765022,0.717855,0.708535,0.691836,...,-1.736215,-1.723609,-1.730770,-1.767129,-1.791136,-1.787874,-1.769603,-1.755394,-1.783720,0
4,0.876973,0.864326,0.801489,0.735828,0.716139,0.703013,0.681590,0.651481,0.649721,0.639855,...,-1.958845,-1.995900,-2.030070,-2.019414,-2.005512,-1.999898,-2.020077,-2.048122,-2.082585,0


In [3]:
GROUP = [RAMAN_PROCESSED[RAMAN_PROCESSED["label"] == i].reset_index(drop=True) for i in range(RAMAN_PROCESSED["label"].max() + 1)]

In [4]:
mat = io.loadmat("../../data/Matlab codes/data.mat")
TRANSCRIPTOME = pd.DataFrame(np.mean(mat["spombe_transcriptome"], axis=2))
TRANSCRIPTOME["label"] = range(TRANSCRIPTOME.shape[0])
TRANSCRIPTOME.to_csv("../../data/TRANSCRIPTOME.csv", index=False)
TRANSCRIPTOME.head()

/home/katori/anaconda3/envs/raman/lib/python3.11/site-packages/scipy/io/matlab/_mio.py:227: MatReadWarning: Duplicate variable name "None" in stream - replacing previous with new
Consider mio5.varmats_from_mat to split file into single variable files
  matfile_dict = MR.get_variables(variable_names)


,0,1,2,3,4,5,6,7,8,9,...,6551,6552,6553,6554,6555,6556,6557,6558,6559,label
0,19.225850,29.38335,79.48800,32.80480,7.101195,1.21101,87.64890,23.231250,160.2995,4.799190,...,31606.09,35050.050,52729.10,13152.785,6155.145,194.078,8268.355,276840.0,55582.95,0
1,18.258500,25.59710,92.78095,34.47605,6.667625,0.74667,74.50975,22.126050,222.6860,9.638689,...,18407.50,42399.185,60035.20,13023.200,4271.420,263.221,11688.100,396968.0,48771.65,1
2,19.978850,21.57395,77.10295,27.37350,20.463750,0.84722,56.34510,8.691435,193.8480,17.956250,...,22123.35,90745.950,94191.15,25344.400,8367.400,0.000,22223.800,1052662.5,91330.85,2
3,22.185055,38.24995,104.96795,41.98480,4.733160,0.00000,71.33305,16.772700,196.4020,5.328506,...,0.00,74301.000,118475.45,27993.150,8382.475,0.000,17990.795,312665.5,96659.45,3
4,11.566850,23.87830,101.44900,40.48195,10.518160,0.00000,55.27985,16.372900,299.4645,7.228260,...,0.00,60735.750,88174.75,19185.900,5774.665,0.000,16664.150,545737.5,52562.45,4


In [5]:
clusterNum = TRANSCRIPTOME.shape[0]
colorList = ["gray", "#B51700"]
nameList = ["PCA", "NRM"]

## Perform transcriptomic profile estimation

In [6]:
cutPercentages = np.arange(80, 100, 0.5)
cutPercentages

array([80. , 80.5, 81. , 81.5, 82. , 82.5, 83. , 83.5, 84. , 84.5, 85. ,
       85.5, 86. , 86.5, 87. , 87.5, 88. , 88.5, 89. , 89.5, 90. , 90.5,
       91. , 91.5, 92. , 92.5, 93. , 93.5, 94. , 94.5, 95. , 95.5, 96. ,
       96.5, 97. , 97.5, 98. , 98.5, 99. , 99.5])

In [7]:
predict_list = []
percent_to_dim_PCA = []
percent_NRM = []

for cutPercent in cutPercentages:
    print(".", end="")
    raman_model = Raman_model(RAMAN_PROCESSED, cutRange=cutPercent, cutMode="percent_fixedDim")
    raman_model.calcTransformation()

    out = []
    for DATA in [raman_model.RAMAN_PCA, raman_model.RAMAN_NRM]:
        lda_model = LDA_model()
        DATA_LDA = lda_model.fit_transform(DATA)

        DATA_LDA = DATA_LDA.groupby("label").mean()
        DATA_LDA["label"] = np.arange(DATA_LDA.shape[0])

        PREDICT = calcPrediction(TRANSCRIPTOME, DATA_LDA, n_components=0, max_iter=50000)

        out.append(PREDICT)

    predict_list.append(out)
    percent_to_dim_PCA.append(raman_model.k_hat)
    percent_NRM.append(raman_model.percent_tilde)

........................................

In [19]:
SUMMARY_TABLE = pd.DataFrame(columns=[f"{i:.1f}%" for i in cutPercentages]).T
SUMMARY_TABLE["dim_PCA"] = percent_to_dim_PCA
SUMMARY_TABLE["percent_NRM"] = percent_NRM

out_pca = np.array([np.sum((returnValues(data[0]) - returnValues(TRANSCRIPTOME)) ** 2, axis=1).sum()
                    for data in predict_list])

out_nrm = np.array([np.sum((returnValues(data[1]) - returnValues(TRANSCRIPTOME)) ** 2, axis=1).sum()
                    for data in predict_list])
    

SUMMARY_TABLE["PRESS_PCA"] = out_pca
SUMMARY_TABLE["PRESS_NRM"] = out_nrm
SUMMARY_TABLE["PRESS_diff"] = out_nrm - out_pca

out_pca_perCon = np.vstack([np.sum((returnValues(data[0]) - returnValues(TRANSCRIPTOME)) ** 2, axis=1)
                            for data in predict_list])

out_nrm_perCon = np.vstack([np.sum((returnValues(data[1]) - returnValues(TRANSCRIPTOME)) ** 2, axis=1)
                            for data in predict_list])

SUMMARY_TABLE.loc[:, [f"PRESS_PCA_c{i + 1}" for i in range(clusterNum)]] = out_pca_perCon
SUMMARY_TABLE.loc[:, [f"PRESS_NRM_c{i + 1}" for i in range(clusterNum)]] = out_nrm_perCon


out_pca = np.array([np.hstack([cosine_similarity(a.reshape(1, -1), b.reshape(1, -1))[0]
                               for a, b in zip(returnValues(data[0]), returnValues(TRANSCRIPTOME))]).mean()
                    for data in predict_list])

out_nrm = np.array([np.hstack([cosine_similarity(a.reshape(1, -1), b.reshape(1, -1))[0]
                               for a, b in zip(returnValues(data[1]), returnValues(TRANSCRIPTOME))]).mean()
                    for data in predict_list])

SUMMARY_TABLE["cosine_PCA"] = out_pca
SUMMARY_TABLE["cosine_NRM"] = out_nrm

SUMMARY_TABLE["cosine_diff"] = out_nrm - out_pca

SUMMARY_TABLE.head()

,dim_PCA,percent_NRM,PRESS_PCA,PRESS_NRM,PRESS_diff,PRESS_PCA_c1,PRESS_PCA_c2,PRESS_PCA_c3,PRESS_PCA_c4,PRESS_PCA_c5,...,PRESS_NRM_c4,PRESS_NRM_c5,PRESS_NRM_c6,PRESS_NRM_c7,PRESS_NRM_c8,PRESS_NRM_c9,PRESS_NRM_c10,cosine_PCA,cosine_NRM,cosine_diff
80.0%,3,84.340141,1.771563e+12,1.771551e+12,-1.124740e+07,4.513867e+10,3.953855e+10,6.374599e+11,2.401069e+11,1.017208e+11,...,2.401002e+11,1.017211e+11,1.186948e+11,6.454892e+10,5.706016e+10,3.921923e+10,4.280780e+11,0.909213,0.909214,2.376565e-07
80.5%,3,84.340141,1.771563e+12,1.771551e+12,-1.124740e+07,4.513867e+10,3.953855e+10,6.374599e+11,2.401069e+11,1.017208e+11,...,2.401002e+11,1.017211e+11,1.186948e+11,6.454892e+10,5.706016e+10,3.921923e+10,4.280780e+11,0.909213,0.909214,2.376565e-07
81.0%,3,84.340141,1.771563e+12,1.771551e+12,-1.124740e+07,4.513867e+10,3.953855e+10,6.374599e+11,2.401069e+11,1.017208e+11,...,2.401002e+11,1.017211e+11,1.186948e+11,6.454892e+10,5.706016e+10,3.921923e+10,4.280780e+11,0.909213,0.909214,2.376565e-07
81.5%,3,84.340141,1.771563e+12,1.771551e+12,-1.124740e+07,4.513867e+10,3.953855e+10,6.374599e+11,2.401069e+11,1.017208e+11,...,2.401002e+11,1.017211e+11,1.186948e+11,6.454892e+10,5.706016e+10,3.921923e+10,4.280780e+11,0.909213,0.909214,2.376565e-07
82.0%,3,84.340141,1.771563e+12,1.771551e+12,-1.124740e+07,4.513867e+10,3.953855e+10,6.374599e+11,2.401069e+11,1.017208e+11,...,2.401002e+11,1.017211e+11,1.186948e+11,6.454892e+10,5.706016e+10,3.921923e+10,4.280780e+11,0.909213,0.909214,2.376565e-07


In [20]:
SUMMARY_TABLE.to_csv(f"../../results/SUMMARY_PERCENT_fixedDim_SPOMBE.csv", index=True)